In [15]:
import os
os.environ['MPLBACKEND'] = 'Qt5Agg'  # Set backend before importing matplotlib

import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('Qt5Agg')  # Explicitly set backend
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import chi2_contingency

# Read the CSV file
df = pd.read_csv('E:/plotsNPX/firingRateGroups.csv')

drop = ['Unnamed: 0', 'Unnamed: 1']
# Remove the first column (index column) if it exists
if 'Unnamed: 0' in df.columns:
    df = df.drop('Unnamed: 0', axis=1)

# Display the first few rows to verify the structure
print("DataFrame structure:")
print(df.head())
print(f"\nDataFrame shape: {df.shape}")
print(f"Column names: {list(df.columns)}")

# Reshape data for ANOVA (long format)
anova_data = []
for i in range(len(df)):  # For each neuron
    for j, condition in enumerate(['470first', '620first', 'Simultaneous']):
        # Group 1 data (columns 0, 1, 2)
        firing_rate_group1 = df.iloc[i, j]
        if firing_rate_group1 > 0:  # Only include active responses
            anova_data.append({
                'neuron_id': i,
                'condition': condition,
                'group': 'Group1',
                'firing_rate': firing_rate_group1
            })
        
        # Group 2 data (columns 3, 4, 5)
        firing_rate_group2 = df.iloc[i, j + 3]
        if firing_rate_group2 > 0:  # Only include active responses
            anova_data.append({
                'neuron_id': i,
                'condition': condition,
                'group': 'Group2',
                'firing_rate': firing_rate_group2
            })

anova_df = pd.DataFrame(anova_data)

print(f"\nANOVA dataset shape: {anova_df.shape}")
print(f"Active responses: {len(anova_df)}")

# Two-Way ANOVA using statsmodels
model = ols('firing_rate ~ C(condition) + C(group) + C(condition):C(group)', data=anova_df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

print("\nTWO-WAY ANOVA RESULTS:")
print("=" * 50)
print(anova_table)
print("=" * 50)

# Post-hoc tests for significant effects
# Post-hoc for condition
if anova_table['PR(>F)']['C(condition)'] < 0.05:
    print("\nPOST-HOC TESTS FOR CONDITION:")
    tukey_condition = pairwise_tukeyhsd(anova_df['firing_rate'], anova_df['condition'], alpha=0.05)
    print(tukey_condition)

# Post-hoc for group
if anova_table['PR(>F)']['C(group)'] < 0.05:
    print("\nPOST-HOC TESTS FOR GROUP:")
    tukey_group = pairwise_tukeyhsd(anova_df['firing_rate'], anova_df['group'], alpha=0.05)
    print(tukey_group)

# Calculate and print effect sizes
def eta_squared(ss_effect, ss_total):
    return ss_effect / ss_total

ss_condition = anova_table['sum_sq']['C(condition)']
ss_group = anova_table['sum_sq']['C(group)']
ss_interaction = anova_table['sum_sq']['C(condition):C(group)']
ss_total = ss_condition + ss_group + ss_interaction + anova_table['sum_sq']['Residual']

print(f"\nEFFECT SIZES (η²):")
print(f"Condition: {eta_squared(ss_condition, ss_total):.3f}")
print(f"Group: {eta_squared(ss_group, ss_total):.3f}")
print(f"Interaction: {eta_squared(ss_interaction, ss_total):.3f}")

# Create visualization
plt.figure(figsize=(14, 6))

# Subplot 1: Mean firing rates
plt.subplot(1, 2, 1)
conditions = ['470first', '620first', 'Simultaneous']

# Calculate means for active neurons only
group1_means = []
group2_means = []
for j, condition in enumerate(conditions):
    # Group 1 means
    group1_data = df.iloc[:, j]
    group1_active = group1_data[group1_data > 0]
    group1_means.append(group1_active.mean() if len(group1_active) > 0 else 0)
    
    # Group 2 means
    group2_data = df.iloc[:, j + 3]
    group2_active = group2_data[group2_data > 0]
    group2_means.append(group2_active.mean() if len(group2_active) > 0 else 0)

x = np.arange(len(conditions))
width = 0.35

plt.bar(x - width/2, group1_means, width, label='Group 1', alpha=0.8, color='gray')
plt.bar(x + width/2, group2_means, width, label='Group 2', alpha=0.8, color='black')

plt.xlabel('Condition')
plt.ylabel('Mean Firing Rate (Hz)')
plt.title('Firing Rates by Condition and Group\n(Active neurons only)')
plt.xticks(x, conditions)
plt.legend()
plt.grid(True, alpha=0.3)

# Add value labels on bars
for i, v in enumerate(group1_means):
    plt.text(i - width/2, v + 0.1, f'{v:.2f}', ha='center', va='bottom')
for i, v in enumerate(group2_means):
    plt.text(i + width/2, v + 0.1, f'{v:.2f}', ha='center', va='bottom')

# Subplot 2: Response prevalence
plt.subplot(1, 2, 2)
response_rates_group1 = []
response_rates_group2 = []

for j, condition in enumerate(conditions):
    # Group 1 response rate
    group1_data = df.iloc[:, j]
    response_rate_group1 = (group1_data > 0).mean() * 100
    response_rates_group1.append(response_rate_group1)
    
    # Group 2 response rate
    group2_data = df.iloc[:, j + 3]
    response_rate_group2 = (group2_data > 0).mean() * 100
    response_rates_group2.append(response_rate_group2)

plt.bar(x - width/2, response_rates_group1, width, label='Group 1', alpha=0.8, color='gray')
plt.bar(x + width/2, response_rates_group2, width, label='Group 2', alpha=0.8, color='black')

plt.xlabel('Condition')
plt.ylabel('Response Rate (%)')
plt.title('Percentage of Neurons Responding')
plt.xticks(x, conditions)
plt.legend()
plt.grid(True, alpha=0.3)

# Add value labels on bars
for i, v in enumerate(response_rates_group1):
    plt.text(i - width/2, v + 1, f'{v:.1f}%', ha='center', va='bottom')
    plt.text(i + width/2, v + 1, f'{v:.1f}%', ha='center', va='bottom')

plt.tight_layout()
plt.show()

# Chi-square tests for response prevalence
print("\nCHI-SQUARE TESTS FOR RESPONSE PREVALENCE:")
for j, condition in enumerate(['470first', '620first', 'Simultaneous']):
    # Group 1 data
    group1_data = df.iloc[:, j]
    responded_group1 = (group1_data > 0).sum()
    no_response_group1 = len(group1_data) - responded_group1
    
    # Group 2 data
    group2_data = df.iloc[:, j + 3]
    responded_group2 = (group2_data > 0).sum()
    no_response_group2 = len(group2_data) - responded_group2
    
    contingency_table = [[responded_group1, no_response_group1],
                        [responded_group2, no_response_group2]]
    
    chi2, p, dof, expected = chi2_contingency(contingency_table)
    
    print(f"{condition}:")
    print(f"  χ²({dof}) = {chi2:.3f}, p = {p:.3f}")
    print(f"  Group 1: {responded_group1}/53 ({responded_group1/53*100:.1f}%) responded")
    print(f"  Group 2: {responded_group2}/53 ({responded_group2/53*100:.1f}%) responded")
    if p < 0.05:
        print(f"  → SIGNIFICANT DIFFERENCE (p < 0.05)")
    print()

# Additional summary statistics
print("SUMMARY STATISTICS:")
print("=" * 40)
for j, condition in enumerate(conditions):
    group1_active = df.iloc[:, j][df.iloc[:, j] > 0]
    group2_active = df.iloc[:, j + 3][df.iloc[:, j + 3] > 0]
    
    print(f"{condition}:")
    print(f"  Group 1: {len(group1_active)} active neurons, Mean = {group1_active.mean():.3f} ± {group1_active.std():.3f} Hz")
    print(f"  Group 2: {len(group2_active)} active neurons, Mean = {group2_active.mean():.3f} ± {group2_active.std():.3f} Hz")
    print()

DataFrame structure:
   470first SP  620first SP  Simultaneous SP  470first TR  620first TR  \
0          0.0          0.0              5.6      0.00000      0.00000   
1         16.4         19.2             19.6     14.66667     18.66667   
2         13.6         21.6             17.2     14.66667     26.00000   
3          0.0          0.0              0.4      0.00000      0.00000   
4          0.0          0.0              0.0      0.00000      0.00000   

   Simultaneous TR  
0         0.000000  
1        10.666670  
2        17.333330  
3         0.666667  
4         0.000000  

DataFrame shape: (54, 6)
Column names: ['470first SP', '620first SP', 'Simultaneous SP', '470first TR', '620first TR', 'Simultaneous TR']

ANOVA dataset shape: (188, 4)
Active responses: 188

TWO-WAY ANOVA RESULTS:
                            sum_sq     df         F    PR(>F)
C(condition)              8.694710    2.0  0.205242  0.814638
C(group)                  9.229658    1.0  0.435740  0.510021
C(cond

In [14]:
import os
os.environ['MPLBACKEND'] = 'Qt5Agg'  # Set backend before importing matplotlib

import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('Qt5Agg')  # Explicitly set backend
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import chi2_contingency
import warnings
warnings.filterwarnings('ignore')

# Read the CSV file
csv_path = 'E:/plotsNPX/firingRateGroups.csv'
try:
    df = pd.read_csv(csv_path)
except FileNotFoundError:
    print(f"Error: File not found at '{csv_path}'. Please check the path and ensure the file exists.")
    # Optionally, you can stop further execution
    import sys
    sys.exit(1)

# Remove the first column (index column) if it exists
if 'Unnamed: 0' in df.columns:
    df = df.drop('Unnamed: 0', axis=1)

# Display initial data info
print("DataFrame structure:")
print(df.head())
print(f"\nDataFrame shape: {df.shape}")
print(f"Column names: {list(df.columns)}")
print(f"Data types:\n{df.dtypes}")

# Check for non-numeric values
print("\nChecking for non-numeric values:")
for col in df.columns:
    non_numeric = df[col].apply(lambda x: not isinstance(x, (int, float, np.number)) and not pd.isna(x))
    if non_numeric.any():
        print(f"Column {col} has non-numeric values: {df[col][non_numeric].unique()}")

# Convert all columns to numeric, replacing non-numeric values with NaN
print("\nConverting to numeric values...")
for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Fill NaN values with 0 (assuming no response = 0 firing rate)
df = df.fillna(0)

print(f"After conversion - Data types:\n{df.dtypes}")
print(f"Any remaining NaN values: {df.isna().any().any()}")

# Verify we have the expected number of columns (6)
if len(df.columns) < 6:
    print(f"Warning: Expected 6 columns but found {len(df.columns)}")
    print("Please check your CSV file structure")
    exit()

# Reshape data for ANOVA (long format)
anova_data = []
conditions = ['470first', '620first', 'Simultaneous']

for i in range(len(df)):  # For each neuron
    for j, condition in enumerate(conditions):
        # Group 1 data (columns 0, 1, 2)
        firing_rate_group1 = df.iloc[i, j]
        if firing_rate_group1 > 0:  # Only include active responses
            anova_data.append({
                'neuron_id': i,
                'condition': condition,
                'group': 'Group1',
                'firing_rate': firing_rate_group1
            })
        
        # Group 2 data (columns 3, 4, 5)
        firing_rate_group2 = df.iloc[i, j + 3]
        if firing_rate_group2 > 0:  # Only include active responses
            anova_data.append({
                'neuron_id': i,
                'condition': condition,
                'group': 'Group2',
                'firing_rate': firing_rate_group2
            })

anova_df = pd.DataFrame(anova_data)

print(f"\nANOVA dataset shape: {anova_df.shape}")
print(f"Active responses: {len(anova_df)}")

# Check if we have enough data for ANOVA
if len(anova_df) < 10:
    print("Warning: Very few active responses. Consider including zero responses or check your data.")

# Display sample of ANOVA data
print("\nSample of ANOVA dataset:")
print(anova_df.head(10))

# Check data distribution by group and condition
print("\nData distribution:")
print(anova_df.groupby(['condition', 'group']).agg({
    'firing_rate': ['count', 'mean', 'std']
}).round(3))

# Two-Way ANOVA using statsmodels
try:
    model = ols('firing_rate ~ C(condition) + C(group) + C(condition):C(group)', data=anova_df).fit()
    anova_table = sm.stats.anova_lm(model, typ=2)
    
    print("\nTWO-WAY ANOVA RESULTS:")
    print("=" * 50)
    print(anova_table)
    print("=" * 50)
    
    # Post-hoc tests for significant effects
    # Post-hoc for condition
    if anova_table['PR(>F)']['C(condition)'] < 0.05:
        print("\nPOST-HOC TESTS FOR CONDITION:")
        tukey_condition = pairwise_tukeyhsd(anova_df['firing_rate'], anova_df['condition'], alpha=0.05)
        print(tukey_condition)
    
    # Post-hoc for group
    if anova_table['PR(>F)']['C(group)'] < 0.05:
        print("\nPOST-HOC TESTS FOR GROUP:")
        tukey_group = pairwise_tukeyhsd(anova_df['firing_rate'], anova_df['group'], alpha=0.05)
        print(tukey_group)
    
    # Calculate and print effect sizes
    def eta_squared(ss_effect, ss_total):
        return ss_effect / ss_total
    
    ss_condition = anova_table['sum_sq']['C(condition)']
    ss_group = anova_table['sum_sq']['C(group)']
    ss_interaction = anova_table['sum_sq']['C(condition):C(group)']
    ss_total = ss_condition + ss_group + ss_interaction + anova_table['sum_sq']['Residual']
    
    print(f"\nEFFECT SIZES (η²):")
    print(f"Condition: {eta_squared(ss_condition, ss_total):.3f}")
    print(f"Group: {eta_squared(ss_group, ss_total):.3f}")
    print(f"Interaction: {eta_squared(ss_interaction, ss_total):.3f}")

except Exception as e:
    print(f"Error in ANOVA analysis: {e}")
    print("This might be due to insufficient data or other issues.")

# Create visualization
plt.figure(figsize=(15, 10))

# Subplot 1: Mean firing rates (only active neurons)
plt.subplot(2, 2, 1)
conditions = ['470first', '620first', 'Simultaneous']

# Calculate means for active neurons only
group1_means = []
group2_means = []
group1_sems = []  # Standard error of mean
group2_sems = []

for j, condition in enumerate(conditions):
    # Group 1 means
    group1_data = df.iloc[:, j]
    group1_active = group1_data[group1_data > 0]
    if len(group1_active) > 0:
        group1_means.append(group1_active.mean())
        group1_sems.append(group1_active.sem())
    else:
        group1_means.append(0)
        group1_sems.append(0)
    
    # Group 2 means
    group2_data = df.iloc[:, j + 3]
    group2_active = group2_data[group2_data > 0]
    if len(group2_active) > 0:
        group2_means.append(group2_active.mean())
        group2_sems.append(group2_active.sem())
    else:
        group2_means.append(0)
        group2_sems.append(0)

x = np.arange(len(conditions))
width = 0.35

bars1 = plt.bar(x - width/2, group1_means, width, yerr=group1_sems, 
               label='Group 1', alpha=0.8, color='gray', capsize=5)
bars2 = plt.bar(x + width/2, group2_means, width, yerr=group2_sems,
               label='Group 2', alpha=0.8, color='black', capsize=5)

plt.xlabel('Condition')
plt.ylabel('Mean Firing Rate (Hz)')
plt.title('Firing Rates by Condition and Group\n(Active neurons only)')
plt.xticks(x, conditions, rotation=45)
plt.legend()
plt.grid(True, alpha=0.3)

# Add value labels on bars
for i, (v, sem) in enumerate(zip(group1_means, group1_sems)):
    plt.text(i - width/2, v + sem + 0.1, f'{v:.2f}', ha='center', va='bottom', fontsize=9)
for i, (v, sem) in enumerate(zip(group2_means, group2_sems)):
    plt.text(i + width/2, v + sem + 0.1, f'{v:.2f}', ha='center', va='bottom', fontsize=9)

if len(anova_df) > 0:
    for i, group in enumerate(['Group1', 'Group2']):
        group_data = anova_df[anova_df['group'] == group]
        for j, condition in enumerate(conditions):
            condition_data = group_data[group_data['condition'] == condition]
            if len(condition_data) > 0:
                y_pos = j + (i * 0.1) - 0.05  # Slight offset for different groups
                plt.scatter([y_pos] * len(condition_data), condition_data['firing_rate'], 
                           alpha=0.6, s=30, label=f'{group}' if j == 0 else "")
    
    plt.xlabel('Condition')
    plt.ylabel('Firing Rate (Hz)')
    plt.title('Raw Data Points')
    plt.xticks(range(len(conditions)), conditions)
    plt.legend()
    plt.grid(True, alpha=0.3)

plt.tight_layout()

# Subplot 2: Response prevalence
plt.subplot(2, 2, 2)
response_rates_group1 = []
response_rates_group2 = []

for j, condition in enumerate(conditions):
    # Group 1 response rate
    group1_data = df.iloc[:, j]
    response_rate_group1 = (group1_data > 0).mean() * 100
    response_rates_group1.append(response_rate_group1)
    
    # Group 2 response rate
    group2_data = df.iloc[:, j + 3]
    response_rate_group2 = (group2_data > 0).mean() * 100
    response_rates_group2.append(response_rate_group2)

plt.bar(x - width/2, response_rates_group1, width, label='Group 1', alpha=0.8, color='gray')
plt.bar(x + width/2, response_rates_group2, width, label='Group 2', alpha=0.8, color='black')

plt.xlabel('Condition')
plt.ylabel('Response Rate (%)')
plt.title('Percentage of Neurons Responding')
plt.xticks(x, conditions, rotation=45)
plt.legend()
plt.grid(True, alpha=0.3)

# Add value labels on bars
for i, v in enumerate(response_rates_group1):
    plt.text(i - width/2, v + 1, f'{v:.1f}%', ha='center', va='bottom', fontsize=9)
for i, v in enumerate(response_rates_group2):
    plt.text(i + width/2, v + 1, f'{v:.1f}%', ha='center', va='bottom', fontsize=9)

# Subplot 3: Box plots for firing rates
plt.subplot(2, 2, 3)
if len(anova_df) > 0:
    # Create box plots
    groups_conditions = []
    firing_rates = []
    
    for group in ['Group1', 'Group2']:
        for condition in conditions:
            subset = anova_df[(anova_df['group'] == group) & (anova_df['condition'] == condition)]
            if len(subset) > 0:
                groups_conditions.extend([f"{group}\n{condition}"] * len(subset))
                firing_rates.extend(subset['firing_rate'].values)
    
    if len(firing_rates) > 0:
        # Create unique positions for each group-condition combination
        unique_labels = list(set(groups_conditions))
        positions = []
        labels_for_plot = []
        data_for_plot = []
        
        for label in unique_labels:
            positions.append(len(labels_for_plot))
            labels_for_plot.append(label)
            data_for_plot.append([rate for rate, lbl in zip(firing_rates, groups_conditions) if lbl == label])
        
        plt.boxplot(data_for_plot, positions=positions, labels=labels_for_plot)
        plt.xticks(rotation=45)
        plt.ylabel('Firing Rate (Hz)')
        plt.title('Distribution of Firing Rates')
        plt.grid(True, alpha=0.3)



# Chi-square tests for response prevalence
print("\nCHI-SQUARE TESTS FOR RESPONSE PREVALENCE:")
print("=" * 50)
for j, condition in enumerate(conditions):
    # Group 1 data
    group1_data = df.iloc[:, j]
    responded_group1 = (group1_data > 0).sum()
    no_response_group1 = len(group1_data) - responded_group1
    
    # Group 2 data
    group2_data = df.iloc[:, j + 3]
    responded_group2 = (group2_data > 0).sum()
    no_response_group2 = len(group2_data) - responded_group2
    
    contingency_table = [[responded_group1, no_response_group1],
                        [responded_group2, no_response_group2]]
    
    try:
        chi2, p, dof, expected = chi2_contingency(contingency_table)
        
        print(f"{condition}:")
        print(f"  χ²({dof}) = {chi2:.3f}, p = {p:.3f}")
        print(f"  Group 1: {responded_group1}/{len(group1_data)} ({responded_group1/len(group1_data)*100:.1f}%) responded")
        print(f"  Group 2: {responded_group2}/{len(group2_data)} ({responded_group2/len(group2_data)*100:.1f}%) responded")
        if p < 0.05:
            print(f"  → SIGNIFICANT DIFFERENCE (p < 0.05)")
        else:
            print(f"  → No significant difference")
        print()
    except Exception as e:
        print(f"  Error in chi-square test for {condition}: {e}")

# Additional summary statistics
print("\nSUMMARY STATISTICS:")
print("=" * 40)
for j, condition in enumerate(conditions):
    group1_all = df.iloc[:, j]
    group2_all = df.iloc[:, j + 3]
    group1_active = group1_all[group1_all > 0]
    group2_active = group2_all[group2_all > 0]
    
    print(f"{condition}:")
    print(f"  Group 1: {len(group1_active)}/{len(group1_all)} active neurons")
    if len(group1_active) > 0:
        print(f"    Mean = {group1_active.mean():.3f} ± {group1_active.std():.3f} Hz")
    print(f"  Group 2: {len(group2_active)}/{len(group2_all)} active neurons")
    if len(group2_active) > 0:
        print(f"    Mean = {group2_active.mean():.3f} ± {group2_active.std():.3f} Hz")
    print()

print("Analysis completed successfully!")

DataFrame structure:
   470first SP  620first SP  Simultaneous SP  470first TR  620first TR  \
0          0.0          0.0              5.6      0.00000      0.00000   
1         16.4         19.2             19.6     14.66667     18.66667   
2         13.6         21.6             17.2     14.66667     26.00000   
3          0.0          0.0              0.4      0.00000      0.00000   
4          0.0          0.0              0.0      0.00000      0.00000   

   Simultaneous TR  
0         0.000000  
1        10.666670  
2        17.333330  
3         0.666667  
4         0.000000  

DataFrame shape: (54, 6)
Column names: ['470first SP', '620first SP', 'Simultaneous SP', '470first TR', '620first TR', 'Simultaneous TR']
Data types:
470first SP        float64
620first SP        float64
Simultaneous SP    float64
470first TR        float64
620first TR        float64
Simultaneous TR    float64
dtype: object

Checking for non-numeric values:

Converting to numeric values...
After conversi

In [12]:
import os
os.environ['MPLBACKEND'] = 'Qt5Agg'  # Set backend before importing matplotlib


import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('Qt5Agg')  # Explicitly set backend
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings('ignore')

# Assuming your data is already loaded and processed as 'df'
# This code extends your existing analysis

def advanced_neuroscience_analysis(df):
    """
    Comprehensive analysis suite for neuroscience firing rate data
    """
    conditions = ['470first', '620first', 'Simultaneous']
    
    # Create figure with multiple subplots
    fig = plt.figure(figsize=(20, 24))
    
    # ====================
    # 1. HEATMAP OF ALL NEURONS
    # ====================
    plt.subplot(4, 3, 1)
    sns.heatmap(df.values, cmap='viridis', cbar_kws={'label': 'Firing Rate (Hz)'})
    plt.title('Heatmap: All Neurons Across Conditions')
    plt.xlabel('Conditions (0-2: Group1, 3-5: Group2)')
    plt.ylabel('Neuron ID')
    
    # ====================
    # 2. CORRELATION MATRIX
    # ====================
    plt.subplot(4, 3, 2)
    correlation_matrix = df.corr()
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
                square=True, fmt='.2f')
    plt.title('Correlation Matrix Between Conditions')
    
    # ====================
    # 3. VIOLIN PLOTS
    # ====================
    plt.subplot(4, 3, 3)
    
    # Prepare data for violin plot
    violin_data = []
    for j, condition in enumerate(conditions):
        for group_idx, group_name in enumerate(['Group1', 'Group2']):
            col_idx = j + (group_idx * 3)
            active_data = df.iloc[:, col_idx][df.iloc[:, col_idx] > 0]
            for rate in active_data:
                violin_data.append({
                    'Condition': condition,
                    'Group': group_name,
                    'Firing_Rate': rate,
                    'Combined': f"{condition}\n{group_name}"
                })
    
    if violin_data:
        violin_df = pd.DataFrame(violin_data)
        sns.violinplot(data=violin_df, x='Combined', y='Firing_Rate')
        plt.xticks(rotation=45)
        plt.title('Distribution Shapes: Violin Plots')
        plt.ylabel('Firing Rate (Hz)')
    
    # ====================
    # 4. NEURON RESPONSE PROFILES
    # ====================
    plt.subplot(4, 3, 4)
    
    # Calculate response profiles for each neuron
    response_profiles = []
    for i in range(len(df)):
        profile = []
        for j in range(6):  # All 6 conditions
            profile.append(df.iloc[i, j])
        response_profiles.append(profile)
    
    response_profiles = np.array(response_profiles)
    
    # Plot response profiles for neurons with at least one response > 0
    active_neurons = np.any(response_profiles > 0, axis=1)
    active_profiles = response_profiles[active_neurons]
    
    for i, profile in enumerate(active_profiles[:20]):  # Show first 20 active neurons
        plt.plot(range(6), profile, alpha=0.3, color='black')
    
    # Plot mean profile
    mean_profile = np.mean(active_profiles, axis=0)
    plt.plot(range(6), mean_profile, color='red', linewidth=3, label='Mean')
    plt.xlabel('Condition (0-2: Group1, 3-5: Group2)')
    plt.ylabel('Firing Rate (Hz)')
    plt.title('Individual Neuron Response Profiles')
    plt.xticks(range(6), ['470-G1', '620-G1', 'Sim-G1', '470-G2', '620-G2', 'Sim-G2'], rotation=45)
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # ====================
    # 5. PCA ANALYSIS
    # ====================
    plt.subplot(4, 3, 5)
    
    # Prepare data for PCA (only active neurons)
    active_data = response_profiles[active_neurons]
    if len(active_data) > 2:
        scaler = StandardScaler()
        scaled_data = scaler.fit_transform(active_data)
        
        pca = PCA()
        pca_result = pca.fit_transform(scaled_data)
        
        plt.scatter(pca_result[:, 0], pca_result[:, 1], alpha=0.6)
        plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%} variance)')
        plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%} variance)')
        plt.title('PCA: Neuron Response Patterns')
        plt.grid(True, alpha=0.3)
    
    # ====================
    # 6. HIERARCHICAL CLUSTERING
    # ====================
    plt.subplot(4, 3, 6)
    
    if len(active_data) > 2:
        # Compute linkage matrix
        linkage_matrix = linkage(active_data, method='ward')
        
        # Create dendrogram
        dendrogram(linkage_matrix, truncate_mode='level', p=5)
        plt.title('Hierarchical Clustering of Neurons')
        plt.xlabel('Neuron Clusters')
        plt.ylabel('Distance')
    
    # ====================
    # 7. RESPONSE LATENCY/MAGNITUDE RELATIONSHIP
    # ====================
    plt.subplot(4, 3, 7)
    
    # Calculate coefficient of variation for each neuron
    cv_values = []
    mean_responses = []
    
    for i in range(len(df)):
        neuron_responses = df.iloc[i, :].values
        active_responses = neuron_responses[neuron_responses > 0]
        if len(active_responses) > 1:
            cv = np.std(active_responses) / np.mean(active_responses)
            cv_values.append(cv)
            mean_responses.append(np.mean(active_responses))
    
    if cv_values:
        plt.scatter(mean_responses, cv_values, alpha=0.6)
        plt.xlabel('Mean Firing Rate (Hz)')
        plt.ylabel('Coefficient of Variation')
        plt.title('Response Variability vs Magnitude')
        plt.grid(True, alpha=0.3)
        
        # Add correlation line
        if len(cv_values) > 2:
            z = np.polyfit(mean_responses, cv_values, 1)
            p = np.poly1d(z)
            plt.plot(mean_responses, p(mean_responses), "r--", alpha=0.8)
            corr_coef = np.corrcoef(mean_responses, cv_values)[0,1]
            plt.text(0.05, 0.95, f'r = {corr_coef:.3f}', transform=plt.gca().transAxes)
    
    # ====================
    # 8. CUMULATIVE RESPONSE PROBABILITY
    # ====================
    plt.subplot(4, 3, 8)
    
    for j, condition in enumerate(conditions):
        for group_idx, group_name in enumerate(['Group1', 'Group2']):
            col_idx = j + (group_idx * 3)
            active_data = df.iloc[:, col_idx][df.iloc[:, col_idx] > 0]
            if len(active_data) > 0:
                sorted_data = np.sort(active_data)
                y_vals = np.arange(1, len(sorted_data) + 1) / len(sorted_data)
                plt.plot(sorted_data, y_vals, label=f'{condition}-{group_name}', alpha=0.7)
    
    plt.xlabel('Firing Rate (Hz)')
    plt.ylabel('Cumulative Probability')
    plt.title('Cumulative Distribution Functions')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, alpha=0.3)
    
    # ====================
    # 9. POPULATION VECTOR ANALYSIS
    # ====================
    plt.subplot(4, 3, 9)
    
    # Calculate population vectors for each condition
    pop_vectors = []
    for j in range(6):
        condition_data = df.iloc[:, j].values
        pop_vectors.append(condition_data / np.linalg.norm(condition_data + 1e-10))  # Normalize
    
    # Plot population vector similarities
    similarity_matrix = np.zeros((6, 6))
    for i in range(6):
        for j in range(6):
            similarity_matrix[i, j] = np.dot(pop_vectors[i], pop_vectors[j])
    
    sns.heatmap(similarity_matrix, annot=True, cmap='viridis', fmt='.3f',
                xticklabels=['470-G1', '620-G1', 'Sim-G1', '470-G2', '620-G2', 'Sim-G2'],
                yticklabels=['470-G1', '620-G1', 'Sim-G1', '470-G2', '620-G2', 'Sim-G2'])
    plt.title('Population Vector Similarities')
    
    # ====================
    # 10. RESPONSE SELECTIVITY INDEX
    # ====================
    plt.subplot(4, 3, 10)
    
    selectivity_indices = []
    for i in range(len(df)):
        neuron_responses = df.iloc[i, :].values
        if np.sum(neuron_responses) > 0:
            # Calculate selectivity index (max - mean) / (max + mean)
            max_response = np.max(neuron_responses)
            mean_response = np.mean(neuron_responses[neuron_responses > 0])
            if mean_response > 0:
                selectivity = (max_response - mean_response) / (max_response + mean_response)
                selectivity_indices.append(selectivity)
    
    if selectivity_indices:
        plt.hist(selectivity_indices, bins=20, alpha=0.7, edgecolor='black')
        plt.xlabel('Selectivity Index')
        plt.ylabel('Number of Neurons')
        plt.title('Distribution of Response Selectivity')
        plt.axvline(np.mean(selectivity_indices), color='red', linestyle='--', 
                   label=f'Mean: {np.mean(selectivity_indices):.3f}')
        plt.legend()
        plt.grid(True, alpha=0.3)
    
    # ====================
    # 11. SIGNAL-TO-NOISE RATIO
    # ====================
    plt.subplot(4, 3, 11)
    
    snr_values = []
    for i in range(len(df)):
        neuron_responses = df.iloc[i, :].values
        active_responses = neuron_responses[neuron_responses > 0]
        if len(active_responses) > 1:
            signal = np.mean(active_responses)
            noise = np.std(active_responses)
            if noise > 0:
                snr = signal / noise
                snr_values.append(snr)
    
    if snr_values:
        plt.hist(snr_values, bins=20, alpha=0.7, edgecolor='black')
        plt.xlabel('Signal-to-Noise Ratio')
        plt.ylabel('Number of Neurons')
        plt.title('Distribution of Signal-to-Noise Ratios')
        plt.axvline(np.mean(snr_values), color='red', linestyle='--',
                   label=f'Mean SNR: {np.mean(snr_values):.2f}')
        plt.legend()
        plt.grid(True, alpha=0.3)
    
    # ====================
    # 12. RESPONSE RELIABILITY
    # ====================
    plt.subplot(4, 3, 12)
    
    # Compare within-group reliability
    within_group_correlations = []
    
    # Group 1 correlations
    group1_data = df.iloc[:, 0:3]
    for i in range(3):
        for j in range(i+1, 3):
            corr = group1_data.iloc[:, i].corr(group1_data.iloc[:, j])
            if not np.isnan(corr):
                within_group_correlations.append(('Group1', corr))
    
    # Group 2 correlations
    group2_data = df.iloc[:, 3:6]
    for i in range(3):
        for j in range(i+1, 3):
            corr = group2_data.iloc[:, i].corr(group2_data.iloc[:, j])
            if not np.isnan(corr):
                within_group_correlations.append(('Group2', corr))
    
    if within_group_correlations:
        corr_df = pd.DataFrame(within_group_correlations, columns=['Group', 'Correlation'])
        sns.boxplot(data=corr_df, x='Group', y='Correlation')
        plt.title('Within-Group Response Reliability')
        plt.ylabel('Correlation Coefficient')
        plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # ====================
    # STATISTICAL SUMMARY
    # ====================
    print("ADVANCED STATISTICAL ANALYSIS")
    print("=" * 50)
    
    # 1. PCA Summary
    if len(active_data) > 2:
        print(f"PCA Analysis:")
        print(f"  - Number of active neurons: {len(active_data)}")
        print(f"  - Variance explained by first 3 PCs: {sum(pca.explained_variance_ratio_[:3]):.2%}")
        print(f"  - PC1: {pca.explained_variance_ratio_[0]:.2%}")
        print(f"  - PC2: {pca.explained_variance_ratio_[1]:.2%}")
        print(f"  - PC3: {pca.explained_variance_ratio_[2]:.2%}" if len(pca.explained_variance_ratio_) > 2 else "")
        print()
    
    # 2. Clustering Analysis
    if len(active_data) > 3:
        kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
        clusters = kmeans.fit_predict(active_data)
        print(f"K-means Clustering (k=3):")
        unique, counts = np.unique(clusters, return_counts=True)
        for cluster, count in zip(unique, counts):
            print(f"  - Cluster {cluster}: {count} neurons ({count/len(clusters)*100:.1f}%)")
        print()
    
    # 3. Population Statistics
    print("Population Response Statistics:")
    total_neurons = len(df)
    responsive_neurons = np.sum(np.any(df.values > 0, axis=1))
    print(f"  - Total neurons: {total_neurons}")
    print(f"  - Responsive neurons: {responsive_neurons} ({responsive_neurons/total_neurons*100:.1f}%)")
    
    # Response patterns
    response_patterns = []
    for i in range(len(df)):
        pattern = tuple(df.iloc[i, :] > 0)
        response_patterns.append(pattern)
    
    unique_patterns = list(set(response_patterns))
    print(f"  - Unique response patterns: {len(unique_patterns)}")
    print()
    
    # 4. Cross-condition analysis
    print("Cross-condition Analysis:")
    for i, cond1 in enumerate(conditions):
        for j, cond2 in enumerate(conditions):
            if i < j:
                # Group 1 correlation
                corr_g1 = df.iloc[:, i].corr(df.iloc[:, j])
                # Group 2 correlation
                corr_g2 = df.iloc[:, i+3].corr(df.iloc[:, j+3])
                print(f"  {cond1} vs {cond2}:")
                print(f"    Group 1 correlation: {corr_g1:.3f}")
                print(f"    Group 2 correlation: {corr_g2:.3f}")
    
    print("\nAnalysis completed!")

# Example usage (assuming df is your loaded dataframe): 
advanced_neuroscience_analysis(df)

ADVANCED STATISTICAL ANALYSIS
PCA Analysis:
  - Number of active neurons: 27
  - Variance explained by first 3 PCs: 95.45%
  - PC1: 85.53%
  - PC2: 5.99%
  - PC3: 3.92%



ValueError: Expected a 2-dimensional container but got <class 'pandas.core.series.Series'> instead. Pass a DataFrame containing a single row (i.e. single sample) or a single column (i.e. single feature) instead.

In [6]:
import os
os.environ['MPLBACKEND'] = 'Qt5Agg'  # Set backend before importing matplotlib

import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('Qt5Agg')  # Explicitly set backend
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats, signal
from scipy.stats import mannwhitneyu, kruskal
import warnings
warnings.filterwarnings('ignore')

class NeuropixelsAnalyzer:
    def __init__(self, firing_rate_csv, area_mapping_csv, spike_timestamp_csv):
        """
        Initialize the analyzer with your three CSV files
        
        Parameters:
        - firing_rate_csv: path to firing rates (53 neurons x 6 conditions)
        - area_mapping_csv: path to area mapping (area names + cluster IDs)
        - spike_timestamp_csv: path to spike timestamps (cluster_id, timestamp)
        """
        self.load_data(firing_rate_csv, area_mapping_csv, spike_timestamp_csv)
        self.conditions = ['470first', '620first', 'Simultaneous']
        
    def load_data(self, firing_rate_csv, area_mapping_csv, spike_timestamp_csv):
        """Load and process all datasets"""
        
        print("Loading datasets...")
        
        # Load firing rates
        self.firing_rates = pd.read_csv(firing_rate_csv)
        if 'Unnamed: 0' in self.firing_rates.columns:
            self.firing_rates = self.firing_rates.drop('Unnamed: 0', axis=1)
        
        # Convert to numeric
        for col in self.firing_rates.columns:
            self.firing_rates[col] = pd.to_numeric(self.firing_rates[col], errors='coerce')
        self.firing_rates = self.firing_rates.fillna(0)
        
        # Load area mapping
        self.area_mapping = pd.read_csv(area_mapping_csv)
        self.area_mapping.columns = ['brain_area', 'cluster_id']
        
        # Load spike timestamps
        self.spike_data = pd.read_csv(spike_timestamp_csv)
        self.spike_data.columns = ['cluster_id', 'timestamp']
        
        # Filter spike data for clusters of interest
        self.clusters_of_interest = self.area_mapping['cluster_id'].values
        self.aoi_spikes = self.spike_data[
            self.spike_data['cluster_id'].isin(self.clusters_of_interest)
        ].copy()
        
        print(f"✓ Firing rates loaded: {self.firing_rates.shape}")
        print(f"✓ Area mapping loaded: {len(self.area_mapping)} clusters")
        print(f"✓ Spike timestamps loaded: {len(self.spike_data):,} total spikes")
        print(f"✓ Area of interest spikes: {len(self.aoi_spikes):,} spikes")
        print(f"✓ Brain areas: {self.area_mapping['brain_area'].unique()}")
        
    def area_comparison_analysis(self):
        """Compare neural activity across different brain areas"""
        
        plt.figure(figsize=(20, 12))
        
        # Get unique brain areas
        brain_areas = self.area_mapping['brain_area'].unique()
        n_areas = len(brain_areas)
        
        # Color palette for areas
        colors = plt.cm.Set3(np.linspace(0, 1, n_areas))
        
        # 1. FIRING RATES BY BRAIN AREA
        plt.subplot(3, 4, 1)
        area_firing_rates = []
        area_labels = []
        
        print("Debug: Processing brain areas...")
        print(f"Firing rates DataFrame shape: {self.firing_rates.shape}")
        print(f"Firing rates DataFrame columns: {list(self.firing_rates.columns)}")
        
        for area in brain_areas:
            print(f"\nProcessing area: {area}")
            area_clusters = self.area_mapping[self.area_mapping['brain_area'] == area]['cluster_id'].values
            print(f"Clusters in {area}: {area_clusters}")
            
            # FIXED: Instead of using enumerate on clusters_of_interest, 
            # directly find matching rows in firing_rates DataFrame
            
            # Assume the first column of firing_rates contains cluster IDs
            firing_rates_cluster_col = self.firing_rates.columns[0] if len(self.firing_rates.columns) > 0 else None
            
            if firing_rates_cluster_col is not None:
                # Find rows where cluster_id matches area_clusters
                area_mask = self.firing_rates[firing_rates_cluster_col].isin(area_clusters)
                area_indices = self.firing_rates.index[area_mask].tolist()
                print(f"Found indices for {area}: {area_indices}")
            else:
                # If no cluster ID column, use positional matching (original approach but safer)
                area_indices = []
                for cluster in area_clusters:
                    if cluster in self.clusters_of_interest:
                        try:
                            idx = list(self.clusters_of_interest).index(cluster)
                            if idx < len(self.firing_rates):  # Check bounds
                                area_indices.append(idx)
                        except ValueError:
                            print(f"Cluster {cluster} not found in clusters_of_interest")
                print(f"Positional indices for {area}: {area_indices}")
            
            if area_indices:
                # Verify indices are within bounds
                max_idx = max(area_indices)
                if max_idx >= len(self.firing_rates):
                    print(f"WARNING: Index {max_idx} out of bounds for DataFrame with {len(self.firing_rates)} rows")
                    area_indices = [idx for idx in area_indices if idx < len(self.firing_rates)]
                    print(f"Filtered indices: {area_indices}")
                
                if area_indices:  # Check again after filtering
                    try:
                        # Get mean firing rate across all conditions for this area
                        area_data = self.firing_rates.iloc[area_indices, :].values
                        area_mean_rates = np.mean(area_data[area_data > 0]) if np.any(area_data > 0) else 0
                        
                        area_firing_rates.append(area_mean_rates)
                        area_labels.append(area)
                        print(f"Successfully processed {area}: mean rate = {area_mean_rates:.3f}")
                        
                    except Exception as e:
                        print(f"Error processing area {area}: {e}")
                else:
                    print(f"No valid indices found for area {area}")
            else:
                print(f"No clusters found for area {area}")
        
        # Continue with the rest of your plotting code...
        if area_firing_rates:  # Only plot if we have data
            bars = plt.bar(range(len(area_labels)), area_firing_rates, 
                        color=colors[:len(area_labels)], alpha=0.7)
            plt.xlabel('Brain Area')
            plt.ylabel('Mean Firing Rate (Hz)')
            plt.title('Firing Rates by Brain Area')
            plt.xticks(range(len(area_labels)), area_labels, rotation=45)
            
            # Add value labels
            for i, v in enumerate(area_firing_rates):
                plt.text(i, v + max(area_firing_rates)*0.01, f'{v:.1f}', 
                        ha='center', va='bottom')
        else:
            plt.text(0.5, 0.5, 'No data available', ha='center', va='center', transform=plt.gca().transAxes)
            plt.title('Firing Rates by Brain Area - No Data')
        
        # 2. SPIKE COUNT DISTRIBUTION BY AREA
        plt.subplot(3, 4, 2)
        
        for i, area in enumerate(brain_areas):
            area_clusters = self.area_mapping[self.area_mapping['brain_area'] == area]['cluster_id'].values
            area_spikes = self.aoi_spikes[self.aoi_spikes['cluster_id'].isin(area_clusters)]
            
            spike_counts_per_neuron = []
            for cluster in area_clusters:
                cluster_spike_count = len(area_spikes[area_spikes['cluster_id'] == cluster])
                spike_counts_per_neuron.append(cluster_spike_count)
            
            if spike_counts_per_neuron:
                plt.hist(spike_counts_per_neuron, bins=20, alpha=0.6, 
                        label=area, color=colors[i])
        
        plt.xlabel('Spike Count per Neuron')
        plt.ylabel('Frequency')
        plt.title('Spike Count Distribution by Area')
        plt.legend()
        if len(brain_areas) > 0:
            plt.yscale('log')
        
        # 3. RESPONSE PATTERNS BY AREA AND CONDITION
        plt.subplot(3, 4, 3)
        
        area_condition_means = []
        processed_areas = []
        
        for area in brain_areas:
            area_clusters = self.area_mapping[self.area_mapping['brain_area'] == area]['cluster_id'].values
            
            # Use the same fixed logic as above
            firing_rates_cluster_col = self.firing_rates.columns[0] if len(self.firing_rates.columns) > 0 else None
            
            if firing_rates_cluster_col is not None:
                area_mask = self.firing_rates[firing_rates_cluster_col].isin(area_clusters)
                area_indices = self.firing_rates.index[area_mask].tolist()
            else:
                area_indices = []
                for cluster in area_clusters:
                    if cluster in self.clusters_of_interest:
                        try:
                            idx = list(self.clusters_of_interest).index(cluster)
                            if idx < len(self.firing_rates):
                                area_indices.append(idx)
                        except ValueError:
                            continue
            
            # Filter out-of-bounds indices
            area_indices = [idx for idx in area_indices if idx < len(self.firing_rates)]
            
            if area_indices:
                area_means = []
                for j in range(min(6, self.firing_rates.shape[1])):  # Ensure we don't exceed column count
                    condition_data = self.firing_rates.iloc[area_indices, j]
                    active_responses = condition_data[condition_data > 0]
                    mean_response = active_responses.mean() if len(active_responses) > 0 else 0
                    area_means.append(mean_response)
                
                # Pad with zeros if we have fewer than 6 conditions
                while len(area_means) < 6:
                    area_means.append(0)
                    
                area_condition_means.append(area_means)
                processed_areas.append(area)
        
        # Create heatmap only if we have data
        if area_condition_means and processed_areas:
            area_condition_df = pd.DataFrame(area_condition_means, 
                                        index=processed_areas,
                                        columns=['470_G1', '620_G1', 'Sim_G1', 
                                                '470_G2', '620_G2', 'Sim_G2'])
            
            sns.heatmap(area_condition_df, annot=True, fmt='.1f', cmap='viridis',
                    cbar_kws={'label': 'Mean Firing Rate (Hz)'})
            plt.title('Response Patterns by Area & Condition')
            plt.xlabel('Conditions')
            plt.ylabel('Brain Areas')
        else:
            plt.text(0.5, 0.5, 'No data available', ha='center', va='center', transform=plt.gca().transAxes)
            plt.title('Response Patterns - No Data')
        
        # Continue with the rest of your subplots...
        # [Rest of the method remains the same]
        
        plt.tight_layout()
        plt.show()
        
        # Return results
        if 'area_condition_df' in locals():
            return area_condition_df, [], []  # Return empty lists for other metrics if needed
        else:
            return pd.DataFrame(), [], []
    
    def integrated_statistical_analysis(self):
        """Statistical tests comparing areas and conditions"""
        
        print("\n" + "="*70)
        print("INTEGRATED STATISTICAL ANALYSIS")
        print("="*70)
        
        brain_areas = self.area_mapping['brain_area'].unique()
        
        # 1. AREA COMPARISONS
        print("\n1. BRAIN AREA COMPARISONS:")
        print("-" * 40)
        
        # Compare firing rates between areas
        area_firing_data = []
        area_labels = []
        
        for area in brain_areas:
            area_clusters = self.area_mapping[self.area_mapping['brain_area'] == area]['cluster_id'].values
            area_indices = [i for i, cluster in enumerate(self.clusters_of_interest) if cluster in area_clusters]
            
            if area_indices:
                area_responses = self.firing_rates.iloc[area_indices, :].values.flatten()
                active_responses = area_responses[area_responses > 0]
                area_firing_data.extend(active_responses)
                area_labels.extend([area] * len(active_responses))
        
        # Statistical test between areas
        if len(brain_areas) > 1:
            area_groups = []
            for area in brain_areas:
                area_responses = [rate for rate, label in zip(area_firing_data, area_labels) if label == area]
                if area_responses:
                    area_groups.append(area_responses)
            
            if len(area_groups) >= 2 and all(len(group) > 0 for group in area_groups):
                try:
                    h_stat, p_val = kruskal(*area_groups)
                    print(f"Kruskal-Wallis test between areas: H = {h_stat:.3f}, p = {p_val:.3f}")
                    
                    if p_val < 0.05:
                        print("→ SIGNIFICANT differences between brain areas")
                        
                        # Pairwise comparisons
                        print("\nPairwise comparisons:")
                        for i in range(len(brain_areas)):
                            for j in range(i+1, len(brain_areas)):
                                area1, area2 = brain_areas[i], brain_areas[j]
                                group1 = [rate for rate, label in zip(area_firing_data, area_labels) if label == area1]
                                group2 = [rate for rate, label in zip(area_firing_data, area_labels) if label == area2]
                                
                                if len(group1) > 0 and len(group2) > 0:
                                    u_stat, p_val = mannwhitneyu(group1, group2, alternative='two-sided')
                                    print(f"  {area1} vs {area2}: U = {u_stat:.1f}, p = {p_val:.3f}")
                    else:
                        print("→ No significant differences between brain areas")
                except Exception as e:
                    print(f"Error in area comparison: {e}")
        
        # 2. CONDITION-SPECIFIC AREA RESPONSES
        print("\n2. CONDITION-SPECIFIC RESPONSES BY AREA:")
        print("-" * 50)
        
        for condition_idx, condition in enumerate(['470_G1', '620_G1', 'Sim_G1', '470_G2', '620_G2', 'Sim_G2']):
            print(f"\n{condition}:")
            
            area_responses = []
            area_names = []
            
            for area in brain_areas:
                area_clusters = self.area_mapping[self.area_mapping['brain_area'] == area]['cluster_id'].values
                area_indices = [i for i, cluster in enumerate(self.clusters_of_interest) if cluster in area_clusters]
                
                if area_indices:
                    condition_responses = self.firing_rates.iloc[area_indices, condition_idx]
                    active_responses = condition_responses[condition_responses > 0]
                    
                    if len(active_responses) > 0:
                        mean_response = active_responses.mean()
                        responsive_neurons = len(active_responses)
                        total_neurons = len(area_indices)
                        response_rate = responsive_neurons / total_neurons * 100
                        
                        print(f"  {area}: {mean_response:.2f} ± {active_responses.std():.2f} Hz "
                              f"({responsive_neurons}/{total_neurons} neurons, {response_rate:.1f}%)")
                        
                        area_responses.extend(active_responses.values)
                        area_names.extend([area] * len(active_responses))
            
            # Test for area differences within this condition
            if len(brain_areas) > 1:
                condition_area_groups = []
                for area in brain_areas:
                    area_resp = [resp for resp, name in zip(area_responses, area_names) if name == area]
                    if len(area_resp) > 0:
                        condition_area_groups.append(area_resp)
                
                if len(condition_area_groups) >= 2 and all(len(group) > 0 for group in condition_area_groups):
                    try:
                        h_stat, p_val = kruskal(*condition_area_groups)
                        if p_val < 0.05:
                            print(f"  → Significant area differences (H = {h_stat:.2f}, p = {p_val:.3f})")
                        else:
                            print(f"  → No significant area differences (p = {p_val:.3f})")
                    except:
                        print("  → Could not perform statistical test")
        
        # 3. TEMPORAL PROPERTIES BY AREA
        print("\n3. TEMPORAL PROPERTIES BY AREA:")
        print("-" * 40)
        
        for area in brain_areas:
            area_clusters = self.area_mapping[self.area_mapping['brain_area'] == area]['cluster_id'].values
            area_spikes = self.aoi_spikes[self.aoi_spikes['cluster_id'].isin(area_clusters)]
            
            if len(area_spikes) > 100:
                # Calculate temporal metrics
                all_isis = []
                cv_values = []
                
                for cluster in area_clusters:
                    cluster_spikes = area_spikes[area_spikes['cluster_id'] == cluster]['timestamp'].values
                    if len(cluster_spikes) > 2:
                        isis = np.diff(sorted(cluster_spikes))
                        isis = isis[isis > 0]
                        
                        if len(isis) > 0:
                            all_isis.extend(isis * 1000)  # Convert to ms
                            cv = np.std(isis) / np.mean(isis)
                            cv_values.append(cv)
                
                if all_isis and cv_values:
                    median_isi = np.median(all_isis)
                    mean_cv = np.mean(cv_values)
                    
                    print(f"{area}:")
                    print(f"  Median ISI: {median_isi:.1f} ms")
                    print(f"  Mean CV: {mean_cv:.2f} ({'regular' if mean_cv < 1 else 'irregular'})")
                    print(f"  Neurons analyzed: {len([c for c in area_clusters if len(area_spikes[area_spikes['cluster_id']==c]) > 2])}")
    
    def create_summary_report(self):
        """Generate a comprehensive summary report"""
        
        print("\n" + "="*80)
        print("NEUROPIXELS DATA SUMMARY REPORT")
        print("="*80)
        
        # Basic statistics
        total_spikes = len(self.aoi_spikes)
        n_clusters = len(self.clusters_of_interest)
        brain_areas = self.area_mapping['brain_area'].unique()
        recording_duration = (self.aoi_spikes['timestamp'].max() - self.aoi_spikes['timestamp'].min())
        
        print(f"\nRECORDING OVERVIEW:")
        print(f"• Total neurons analyzed: {n_clusters}")
        print(f"• Brain areas: {', '.join(brain_areas)} ({len(brain_areas)} areas)")
        print(f"• Total spikes: {total_spikes:,}")
        print(f"• Recording duration: {recording_duration:.1f} seconds ({recording_duration/60:.1f} minutes)")
        print(f"• Average firing rate: {total_spikes/recording_duration/n_clusters:.2f} Hz per neuron")
        
        # Area breakdown
        print(f"\nAREA BREAKDOWN:")
        for area in brain_areas:
            area_clusters = self.area_mapping[self.area_mapping['brain_area'] == area]['cluster_id'].values
            n_area_clusters = len(area_clusters)
            area_spikes = self.aoi_spikes[self.aoi_spikes['cluster_id'].isin(area_clusters)]
            area_spike_count = len(area_spikes)
            
            print(f"• {area}: {n_area_clusters} neurons, {area_spike_count:,} spikes "
                  f"({area_spike_count/total_spikes*100:.1f}% of total)")
        
        # Condition responses
        print(f"\nCONDITION RESPONSES:")
        condition_names = ['470nm first (G1)', '620nm first (G1)', 'Simultaneous (G1)',
                          '470nm first (G2)', '620nm first (G2)', 'Simultaneous (G2)']
        
        for i, condition in enumerate(condition_names):
            responsive_neurons = (self.firing_rates.iloc[:, i] > 0).sum()
            mean_response = self.firing_rates.iloc[:, i][self.firing_rates.iloc[:, i] > 0].mean()
            response_rate = responsive_neurons / n_clusters * 100
            
            print(f"• {condition}: {responsive_neurons}/{n_clusters} neurons respond "
                  f"({response_rate:.1f}%), mean = {mean_response:.2f} Hz")

# Example usage instructions
print("""
USAGE INSTRUCTIONS:

1. Initialize the analyzer:
   analyzer = NeuropixelsAnalyzer(
       'path/to/firing_rates.csv',
       'path/to/area_mapping.csv', 
       'path/to/spike_timestamps.csv'
   )

2. Run area comparison analysis:
   area_df, burstiness, selectivity = analyzer.area_comparison_analysis()

3. Run statistical analysis:
   analyzer.integrated_statistical_analysis()

4. Generate summary report:
   analyzer.create_summary_report()

EXAMPLE:
analyzer = NeuropixelsAnalyzer(
    'E:/plotsNPX/values4anova.csv',
    'E:/plotsNPX/area_cluster_mapping.csv',
    'E:/plotsNPX/spike_timestamps.csv'
)

analyzer.area_comparison_analysis()
analyzer.integrated_statistical_analysis()
analyzer.create_summary_report()
""")


USAGE INSTRUCTIONS:

1. Initialize the analyzer:
   analyzer = NeuropixelsAnalyzer(
       'path/to/firing_rates.csv',
       'path/to/area_mapping.csv', 
       'path/to/spike_timestamps.csv'
   )

2. Run area comparison analysis:
   area_df, burstiness, selectivity = analyzer.area_comparison_analysis()

3. Run statistical analysis:
   analyzer.integrated_statistical_analysis()

4. Generate summary report:
   analyzer.create_summary_report()

EXAMPLE:
analyzer = NeuropixelsAnalyzer(
    'E:/plotsNPX/values4anova.csv',
    'E:/plotsNPX/area_cluster_mapping.csv',
    'E:/plotsNPX/spike_timestamps.csv'
)

analyzer.area_comparison_analysis()
analyzer.integrated_statistical_analysis()
analyzer.create_summary_report()



In [ ]:
analyzer = NeuropixelsAnalyzer(
    'E:/plotsNPX/firingRateGroups.csv',           # Your firing rate CSV
    'E:/plotsNPX/allClusters.csv',  # Your area mapping CSV  
    'E:/plotsNPX/clustersFiringtimes.csv'       # Your spike timestamp CSV
)

analyzer.area_comparison_analysis()
analyzer.integrated_statistical_analysis()
analyzer.create_summary_report()



Loading datasets...
✓ Firing rates loaded: (53, 6)
✓ Area mapping loaded: 89 clusters
✓ Spike timestamps loaded: 437,483 total spikes
✓ Area of interest spikes: 405,656 spikes
✓ Brain areas: ["'MRN'" "'MB'" "'SCiw'" "'SCig'" "'bsc'" "'root'" "'POST'" "'scwm'"
 "'VISp6b'" "'VISp6a'" "'VISp5'" "'VISp4'" "'VISp2_3'"]
Debug: Processing brain areas...
Firing rates DataFrame shape: (53, 6)
Firing rates DataFrame columns: ['470first SP', '620first SP', 'Simultaneous SP', '470first TR', '620first TR', 'Simultaneous TR']

Processing area: 'MRN'
Clusters in 'MRN': [  8  14  11  10  28  71  75  83  95  99  98 101 106 109]
Found indices for 'MRN': []
No clusters found for area 'MRN'

Processing area: 'MB'
Clusters in 'MB': [113 125 124 129]
Found indices for 'MB': []
No clusters found for area 'MB'

Processing area: 'SCiw'
Clusters in 'SCiw': [144]
Found indices for 'SCiw': []
No clusters found for area 'SCiw'

Processing area: 'SCig'
Clusters in 'SCig': [148 157 159 161 163 170 169 177 180 183 18

IndexError: positional indexers are out-of-bounds

In [ ]:


class NeuropixelsAnalyzer:
    def __init__(self, firing_rate_csv, area_mapping_csv, spike_times_csv):
        """
        Initialize the analyzer with CSV file paths
        """
        print("Loading data files...")
        
        # Load the data
        self.firing_rates = pd.read_csv(firing_rate_csv)
        self.area_mapping = pd.read_csv(area_mapping_csv)
        self.spike_times = pd.read_csv(spike_times_csv)
        
        # Print basic info about loaded data
        print(f"Firing rates shape: {self.firing_rates.shape}")
        print(f"Area mapping shape: {self.area_mapping.shape}")
        print(f"Spike times shape: {self.spike_times.shape}")
        
        # Print column names for debugging
        print("\nFiring rates columns:", list(self.firing_rates.columns))
        print("Area mapping columns:", list(self.area_mapping.columns))
        print("Spike times columns:", list(self.spike_times.columns))
        
        # Define areas of interest
        self.areas_of_interest = ['VISp6b', 'VISp6a', 'VISp5', 'VISp4', 'VISp2_3']
        
        # Get clusters of interest - using first column as cluster ID
        cluster_id_col = self.firing_rates.columns[0]
        self.clusters_of_interest = self.firing_rates[cluster_id_col].tolist()
        
        print(f"\nTotal clusters in firing rate data: {len(self.clusters_of_interest)}")
        print(f"First 10 cluster IDs: {self.clusters_of_interest[:10]}")
        
        # Process area mapping
        self.process_area_mapping()
        
    def process_area_mapping(self):
        """
        Process area mapping to create a lookup dictionary
        """
        # Assume first column is cluster_id and second is area
        cluster_col = self.area_mapping.columns[0]
        area_col = self.area_mapping.columns[1]
        
        self.cluster_to_area = dict(zip(self.area_mapping[cluster_col], 
                                      self.area_mapping[area_col]))
        
        print(f"\nArea mapping created for {len(self.cluster_to_area)} clusters")
        
        # Count clusters per area
        area_counts = {}
        for cluster_id in self.clusters_of_interest:
            area = self.cluster_to_area.get(cluster_id, 'Unknown')
            area_counts[area] = area_counts.get(area, 0) + 1
        
        print("Clusters per area:")
        for area, count in sorted(area_counts.items()):
            print(f"  {area}: {count}")
    
    def area_comparison_analysis(self):
        """
        Compare firing rates across different brain areas with improved error handling
        """
        print("\n" + "="*60)
        print("AREA COMPARISON ANALYSIS")
        print("="*60)
        
        area_firing_rates = []
        area_labels = []
        area_cluster_counts = []
        
        for area in self.areas_of_interest:
            # Find clusters in this area
            area_clusters = [cluster_id for cluster_id in self.clusters_of_interest 
                           if self.cluster_to_area.get(cluster_id) == area]
            
            print(f"\nProcessing area: {area}")
            print(f"Clusters in this area: {len(area_clusters)}")
            
            if not area_clusters:
                print(f"  No clusters found for area {area}")
                continue
                
            # Find indices of these clusters in the firing rate dataframe
            cluster_id_col = self.firing_rates.columns[0]
            area_indices = []
            
            for cluster_id in area_clusters:
                try:
                    # Find the index where cluster_id matches
                    idx = self.firing_rates[self.firing_rates[cluster_id_col] == cluster_id].index
                    if len(idx) > 0:
                        area_indices.extend(idx.tolist())
                    else:
                        print(f"  Warning: Cluster {cluster_id} not found in firing rate data")
                except Exception as e:
                    print(f"  Error finding cluster {cluster_id}: {e}")
            
            print(f"  Valid indices found: {len(area_indices)}")
            print(f"  Indices range: {min(area_indices) if area_indices else 'None'} to {max(area_indices) if area_indices else 'None'}")
            print(f"  DataFrame has {len(self.firing_rates)} rows")
            
            if area_indices:
                try:
                    # Verify indices are within bounds
                    max_idx = max(area_indices)
                    if max_idx >= len(self.firing_rates):
                        print(f"  ERROR: Index {max_idx} is out of bounds for DataFrame with {len(self.firing_rates)} rows")
                        continue
                    
                    # Get numeric columns (exclude first column which is cluster ID)
                    numeric_cols = self.firing_rates.select_dtypes(include=[np.number]).columns
                    
                    # Get firing rate data for this area
                    area_data = self.firing_rates.iloc[area_indices][numeric_cols].values
                    
                    # Calculate mean firing rate across all conditions
                    area_mean_rates = np.mean(area_data[area_data > 0]) if np.any(area_data > 0) else 0
                    
                    area_firing_rates.append(area_mean_rates)
                    area_labels.append(area)
                    area_cluster_counts.append(len(area_indices))
                    
                    print(f"  Mean firing rate: {area_mean_rates:.3f} Hz")
                    
                except Exception as e:
                    print(f"  ERROR processing area {area}: {e}")
                    continue
        
        if len(area_firing_rates) < 2:
            print("\nInsufficient data for area comparison (need at least 2 areas)")
            return
        
        # Create visualization
        plt.figure(figsize=(12, 8))
        
        # Plot 1: Mean firing rates by area
        plt.subplot(2, 2, 1)
        bars = plt.bar(area_labels, area_firing_rates, alpha=0.7, color='skyblue')
        plt.title('Mean Firing Rates by Brain Area')
        plt.ylabel('Firing Rate (Hz)')
        plt.xticks(rotation=45)
        
        # Add value labels on bars
        for bar, rate in zip(bars, area_firing_rates):
            plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
                    f'{rate:.2f}', ha='center', va='bottom')
        
        # Plot 2: Cluster counts by area
        plt.subplot(2, 2, 2)
        plt.bar(area_labels, area_cluster_counts, alpha=0.7, color='lightcoral')
        plt.title('Number of Clusters by Area')
        plt.ylabel('Number of Clusters')
        plt.xticks(rotation=45)
        
        # Plot 3: Firing rate distribution
        plt.subplot(2, 2, 3)
        plt.boxplot(area_firing_rates, labels=area_labels)
        plt.title('Firing Rate Distribution')
        plt.ylabel('Firing Rate (Hz)')
        plt.xticks(rotation=45)
        
        plt.tight_layout()
        plt.show()
        
        # Statistical analysis
        print(f"\nStatistical Summary:")
        print(f"Areas analyzed: {len(area_labels)}")
        print(f"Mean firing rate across areas: {np.mean(area_firing_rates):.3f} ± {np.std(area_firing_rates):.3f} Hz")
        
        # Store results for later use
        self.area_results = {
            'areas': area_labels,
            'firing_rates': area_firing_rates,
            'cluster_counts': area_cluster_counts
        }
    
    def integrated_statistical_analysis(self):
        """
        Perform statistical tests on the area comparison results
        """
        if not hasattr(self, 'area_results'):
            print("No area results available. Run area_comparison_analysis() first.")
            return
        
        print("\n" + "="*60)
        print("STATISTICAL ANALYSIS")
        print("="*60)
        
        rates = self.area_results['firing_rates']
        areas = self.area_results['areas']
        
        # Basic statistics
        print(f"Number of areas: {len(areas)}")
        print(f"Overall mean: {np.mean(rates):.3f} Hz")
        print(f"Overall std: {np.std(rates):.3f} Hz")
        print(f"Range: {np.min(rates):.3f} - {np.max(rates):.3f} Hz")
        
        # Pairwise comparisons if we have enough areas
        if len(rates) >= 2:
            print("\nPairwise comparisons:")
            for i in range(len(areas)):
                for j in range(i+1, len(areas)):
                    diff = rates[i] - rates[j]
                    print(f"{areas[i]} vs {areas[j]}: {diff:.3f} Hz difference")
    
    def create_summary_report(self):
        """
        Create a comprehensive summary report
        """
        print("\n" + "="*60)
        print("SUMMARY REPORT")
        print("="*60)
        
        print(f"Dataset Overview:")
        print(f"  - Total clusters: {len(self.clusters_of_interest)}")
        print(f"  - Areas mapped: {len(set(self.cluster_to_area.values()))}")
        print(f"  - Areas of interest analyzed: {len(self.areas_of_interest)}")
        
        if hasattr(self, 'area_results'):
            print(f"\nArea Analysis Results:")
            for i, area in enumerate(self.area_results['areas']):
                print(f"  - {area}: {self.area_results['firing_rates'][i]:.3f} Hz "
                      f"({self.area_results['cluster_counts'][i]} clusters)")
        
        print("\nAnalysis completed successfully!")

# Usage example with debug information
def run_analysis_with_debug():
    """
    Run the analysis with enhanced debugging
    """
    try:
        analyzer = NeuropixelsAnalyzer(
            'E:/plotsNPX/firingRateGroups.csv',
            'E:/plotsNPX/allClusters.csv', 
            'E:/plotsNPX/clustersFiringtimes.csv'
        )
        
        analyzer.area_comparison_analysis()
        analyzer.integrated_statistical_analysis()
        analyzer.create_summary_report()
        
    except Exception as e:
        print(f"Error during analysis: {e}")
        import traceback
        traceback.print_exc()

# Run the analysis
if __name__ == "__main__":
    run_analysis_with_debug()

Loading data files...
Firing rates shape: (53, 6)
Area mapping shape: (89, 2)
Spike times shape: (437483, 2)

Firing rates columns: ['470first SP', '620first SP', 'Simultaneous SP', '470first TR', '620first TR', 'Simultaneous TR']
Area mapping columns: ["'MRN'", '4']
Spike times columns: ['75', '623']

Total clusters in firing rate data: 53
First 10 cluster IDs: [0.0, 14.66666667, 14.66666667, 0.0, 0.0, 0.0, 11.33333333, 0.0, 0.0, 0.666666667]

Area mapping created for 13 clusters
Clusters per area:
  Unknown: 53

AREA COMPARISON ANALYSIS

Processing area: VISp6b
Clusters in this area: 0
  No clusters found for area VISp6b

Processing area: VISp6a
Clusters in this area: 0
  No clusters found for area VISp6a

Processing area: VISp5
Clusters in this area: 0
  No clusters found for area VISp5

Processing area: VISp4
Clusters in this area: 0
  No clusters found for area VISp4

Processing area: VISp2_3
Clusters in this area: 0
  No clusters found for area VISp2_3

Insufficient data for area